Install julia dependencies

In [ ]:
using Pkg

In [ ]:
Pkg.add(path="https://github.com/jmurga/MKtest.jl")
Pkg.add(["CSV", "Unzip", "DataFrames", "StatsBase", "RCall", "JLD2", "Suppressor","CondaPkg","ProgressMeter"])

In [ ]:
using MKtest, CSV, Unzip, DataFrames, StatsBase, RCall, JLD2, Suppressor,ProgressMeter

In [ ]:
labstorage = "/labstorage/jmurgamoreno/Immune_Adaptation_Atlas_2023/";
path = "/home/jmurgamoreno/Immune_Adaptation_Atlas_2023/";
@rput labstorage;
@rput path;

In [ ]:
function cell_analysis(param;file,data_tgp,rates=nothing)

    @show file
    alpha,sfs,divergence = MKtest.parse_sfs(param,data=data_tgp,gene_list=file)

    a,s,d = MKtest.data_to_poisson(sfs,divergence,param.dac)

    if any(0 .∈ s)
        deleteat!(sfs,findall(0 .∈ s))
        deleteat!(divergence,findall(0 .∈ s))
        deleteat!(alpha,findall(0 .∈ s))
    end

    # Get cell name
    cell_name = replace(split(file,"/")[end],"_control.txt"=>"")
    cell_name = replace(cell_name,"_case.txt"=>"")

    line_name = split(file,"/")[end-2]

    if occursin("case",file)
        c_type = "case"
    else
        c_type = "control"
    end

    folder = replace(file,".txt" => "")
    mkpath(folder)

    out = @suppress begin
        try
            summstat = MKtest.summary_statistics(param,sfs,divergence,h5_file=rates,output_folder= folder,summstat_size=10^5);

            posteriors = MKtest.ABCreg(output_folder=folder,S=length(param.dac),tol=0.025,rm_summaries=true);

            out = MKtest.summary_abc(posteriors,stat="mode");
            insertcols!(out[1],1,:type=>c_type)
            insertcols!(out[1],1,:cell=>cell_name)
            insertcols!(out[1],1,:line=>line_name)

            out
        catch
            DataFrame(),DataFrame()
        end
    end
    return(out)
end


Change paths as needed!

# Estimating analytical rates

In [ ]:
# Be sure you're using multiple threads to estimate the rates
df = MKtest.rates(adap,gH=[200,2000],gL=[1,10],gam_dfe=[-2000,-200],gam_flanking=[-1000,-500],alpha=[0.0,0.9],iterations=10^5,output= labstorage * "abcmk/rates_hpc.jld2");

# Developmental atlas

In [ ]:
lines = readdir(labstorage * "Developmental/ENS_FULL_genelists_wilcox",join=true)

mkpath(labstorage * "/abcmk/Developmental")

mkpath.(labstorage .* "/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/" .* ["datasets","outputs"])

@rput lines

## Bootstrap developmental genes

In [ ]:
genes = CSV.read(labstorage * "annotations/MKdata_may2023.txt",DataFrame,header=false);
orthologs = CSV.read(labstorage * "annotations/mammals_orthologs.txt",DataFrame,header=true);

rename!(orthologs,:mgPropSitesAdapt => :ids);

In [ ]:
df_prot = CSV.read(labstorage * "/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/ensembl_protcoding_genes_v109",DataFrame,header=false)
insertcols!(df_prot,2,:c=>"no")       
# DataFrame to be changed
# Change directory before to run bootstrap
cd("/home/jmurgamoreno/Immune_Adaptation_Atlas_2023//abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/")

In [ ]:
for l in lines

    files = filter(x -> occursin("FULL_ENSEMBL.txt",x) ,readdir(l,join=true))

    for f ∈ files
        @show f

        f_path,f_de = splitdir(f)

        f_de = replace(f_de,"_FULL_ENSEMBL"=>"")
        f_de = split(f_path,"/")[end] * "_" * split(f_de,".")[1]

        # Extracting the top 500 DE genes by cell line.
        df = CSV.read(f,DataFrame,header=false)

        # Get orthologs only
        rename!(df,:Column1=>:ids)

        df_orthologs = innerjoin(df,orthologs,on=:ids,order=:left)

        top_de = first(df_orthologs,500)
        df_prot[in.(df_prot.Column1,[top_de.ids]),:c] .= "yes"

        CSV.write(labstorage * "/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/datasets/" * f_de * ".txt",df_prot,header=false,delim=" ")

        # Run perl bootstrap
        cmd = `bash -c "./run.sh $f_de"`
        run(cmd)
        df_prot[!,:c] .= "no"

    end
end
mv(labstorage * "/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/datasets/",labstorage * "/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/Developmental")
mv(labstorage * "/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/outputs/",labstorage * "/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/Developmental_outputs")


## ABC-MK

In [ ]:
adap = MKtest.parameters(n=661,dac=[2,4,5,10,20,50,200,661],cutoff=[0.0,0.7]);

In [ ]:
files = filter(x-> !isdir(x),readdir(labstorage * "/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/Developmental_outputs/",join=true))

out_dev = []

@showprogress for i in files[1:4]

    abcmk = cell_analysis(adap,file=i,data_tgp=labstorage * "/annotations/MKdata_may2023.txt",rates=labstorage * "/abcmk/rates_hpc.jld2")
    @show size(abcmk[1])

    push!(out_dev,abcmk)
end

In [ ]:
# Save dict results
JLD2.jldopen(path * "/abmkc/results_immune_abc.jld2", "a+") do file
    file["developmental"] = out_dev
end

## Clean and plot

In [ ]:
df_dev = vcat(map(x->x[1],out_dev)...);
df_dev = sort(df_dev,[:line,:cell]);
df_dev.line .= map(x-> join(split(x,"_")[1:2],"_"),df_dev.cell);
df_dev.cell = replace.(df_dev.cell,"HSC_progenitors_"=>"");
df_dev.cell = replace.(df_dev.cell,"MegaK_Ery_"=>"");
df_dev.cell = replace.(df_dev.cell,"Lymphoid_ALL_"=>"");
df_dev.cell = replace.(df_dev.cell,"Myeloid_ALL_"=>"");

df_dev = insertcols(df_dev,16,:n=>0);
df_dev[df_dev.type .== "case",:n] .= countlines.(files[occursin.("case",files)][1:2]);

df_ci_dev = vcat(map(x-> x[2] ,out_dev[occursin.("case",files[1:4])])...);
insertcols!(df_ci_dev,1,:cell=>map(x-> splitdir(x)[end],files[occursin.("case",files)]));

insertcols!(df_ci_dev,1,:line=>map(x-> join(split(splitdir(x)[end],"_")[1:2],"_"),files[occursin.("case",files)]));

df_ci_dev.cell = replace.(df_ci_dev.cell,"HSC_progenitors_"=>"");
df_ci_dev.cell = replace.(df_ci_dev.cell,"MegaK_Ery_"=>"");
df_ci_dev.cell = replace.(df_ci_dev.cell,"Lymphoid_ALL_"=>"");
df_ci_dev.cell = replace.(df_ci_dev.cell,"Myeloid_ALL_"=>"");
df_ci_dev.cell = replace.(df_ci_dev.cell,".txt"=>"");
df_ci_dev.cell = replace.(df_ci_dev.cell,"_case"=>"");
df_ci_dev = sort(df_ci_dev,[:line,:cell]);

df_ci_dev = innerjoin(df_ci_dev,df_dev[df_dev.type .=="case",[:line,:cell,:n]],on=[:line,:cell])


In [ ]:

CSV.write(path  * "/abcmk/Developmental/developmental_abcmk_inference_alpha_top_500_orthologs.txt",df_cell_lines);

CSV.write(path  * "/abcmk/Developmental/developmental_abcmk_ci_alpha_top_500_orthologs.txt",df_ci_cell_lines);

pal = CSV.read("/home/jmurgamoreno/immune_adaptation_atlas/results/cell_lines/line_cell_palette.txt",DataFrame);

df_cell_lines = innerjoin(df_cell_lines,pal,on=[:cell,:line]);

@rput df_cell_lines;
@rput df_ci_cell_lines;
@rput pal;
@rput path

In [ ]:
R"""
library(magrittr)
library(paletteer)
library(data.table)
library(dplyr)
library(tidyr)
library(ggplot2)

empirical_p <- function(df,symbol){
    x = split(df,df$cell)

    out = list()
    for(i in names(x)){
        x[[i]]$pvalue = rank(x[[i]][[symbol]] * -1,na.last = "keep")/sum(is.na (x[[i]][[symbol]]) == F)
        x[[i]]$pvalue_strong = rank(x[[i]][[paste0(symbol,"_strong")]] * -1,na.last = "keep")/sum(is.na (x[[i]][[paste0(symbol,"_strong")]]) == F)
        x[[i]]$pvalue_weak = rank(x[[i]][[paste0(symbol,"_weak")]] * -1,na.last = "keep")/sum(is.na (x[[i]][[paste0(symbol,"_weak")]]) == F)
        out[[i]] = x[[i]]
    }
    return(rbindlist(out))
}

df = as.data.table(df_cell_lines)
df$cell = gsub("_case.txt","",df$cell)
df$cell = gsub("_control.txt","",df$cell)
df$cell = gsub("_FULL_ENSEMBL_TOP","",df$cell)
df = df %>% select(-c('rank')) %>% as.data.table
df = df[cell!="MYELOCYTE"]
df = empirical_p(df,"α")

df = df[cell!=line]

df[cell=='all']$cell = paste0(df[cell=='all']$cell,"_",df[cell=='all']$line)

for(i in c("CMP","DNearly_T","DNP_T","EARLY_MK","GMP","MEMP","MEP","PRE_PRO_B","PROMONOCYTE","PROMYELOCYTE")){
    df[cell==i]$cell = paste0(df[cell==i]$cell,"_",df[cell==i]$line)
}

df[cell=='Progenitor']$cell = paste0(df[cell=='Progenitor']$cell,"_",df[cell=='Progenitor']$line)

df_ci = as.data.table(df_ci_cell_lines)
df_ci$cell = gsub("_case.txt","",df_ci$cell)
df_ci$cell = gsub("_control.txt","",df_ci$cell)
df_ci$cell = gsub("_FULL_ENSEMBL_TOP","",df_ci$cell)
df_ci = df_ci[cell!="MYELOCYTE"]

df_ci = df_ci[cell!=line]

df_ci[cell=='all']$cell = paste0(df_ci[cell=='all']$cell,"_",df_ci[cell=='all']$line)

for(i in c("CMP","DNearly_T","DNP_T","EARLY_MK","GMP","MEMP","MEP","PRE_PRO_B","PROMONOCYTE","PROMYELOCYTE")){
    df_ci[cell==i]$cell = paste0(df_ci[cell==i]$cell,"_",df_ci[cell==i]$line)
}

df_ci[cell=='Progenitor']$cell = paste0(df_ci[cell=='Progenitor']$cell,"_",df_ci[cell=='Progenitor']$line)

d = melt(df[,c(1:6,16:19)],id.vars=c("line","cell","subline","type","n","pvalue","color"))

d$cell = gsub("_","-",d$cell)
d$cell = gsub("DNearly","DN(early)",d$cell)
d$cell = gsub("DNP","DN(P)",d$cell)
d$cell = gsub("DNQ","DN(Q)",d$cell)
d$cell = gsub("DPP","DP(P)",d$cell)

d$cell = gsub("-HSC-progenitors"," (HSC-progenitors)",d$cell)
d$cell = gsub("-Myeloid-ALL"," (Myeloid-ALL)",d$cell)

d = d[order(variable,decreasing=T)]

d_p= melt(df[,c(1:3,16:21)],id.vars=c("line","cell","subline","type","n","color"))

d_p$cell = gsub("_","-",d_p$cell)
d_p$cell = gsub("DNearly","DN(early)",d_p$cell)
d_p$cell = gsub("DNP","DN(P)",d_p$cell)
d_p$cell = gsub("DNQ","DN(Q)",d_p$cell)
d_p$cell = gsub("DPP","DP(P)",d_p$cell)
d_p$cell = gsub("-HSC-progenitors"," (HSC-progenitors)",d_p$cell)
d_p$cell = gsub("-Myeloid-ALL"," (Myeloid-ALL)",d_p$cell)

d_p$label=""
d_p[type=='case'] = d_p[type=='case'] %<>% mutate(
        label = case_when(
            value > 0.05 ~ "",
            value > 0.01 ~ "*",
            value > 0.001 ~ "**",
            value > 0.0001 ~ "***",
            value > 0.00001 ~ "****",
            !is.na(value) ~ "",
            TRUE ~ NA_character_
        )
    )

tmp = d[type == "case" & variable == "α"]
tmp = split(tmp,tmp$subline)

subline_order <- c("HSC_progenitors", "Tcells", "Bcells", "Innate", "MACs","Granulocytes","DCs","Stromal","MegaKaryocytes","Erythrocytes")

tmp = tmp[subline_order]

for(i in names(tmp)){
    tmp[[i]] = tmp[[i]][order(value,decreasing=T)]
}

tmp = rbindlist(tmp)
x = tmp$cell

d$variable = factor(d$variable,levels=c("α","α_strong","α_weak"))
d$cell = factor(d$cell,levels=x)
d_p$cell = factor(d_p$cell,levels=x)
d$pvalue = d_p$value
d$label = d_p$label

d_flt = d[!grepl("all",d$cell)]
cols = c("line","cell")
d_flt = d_flt %>% group_by(across(all_of(cols))) %>% mutate("n"=n()) %>% as.data.table
# d_flt = d_flt[n>3]

x_pos = c(12.5,22.5,31.5,36.5,47.5,51.5,58.5,62.5,67.5)
clrs = d_flt[type=='case',c('cell','color')] %>% unique
clrs = clrs[order(cell)]
p = ggplot(d_flt[type=="control"],aes(x=cell,y=value,fill=cell)) +
   geom_violin(width=0.8) +
   geom_errorbar(data=d_flt[type=="case"],aes(x = cell, ymin = value,ymax = value),size=1,color="black") +
   geom_text(data=d_flt[type=="case"],aes(x=cell,y=-0.05,label = label,color=line),size=14,color="red",angle = 90, vjust = 0.75, hjust=1) +
   geom_vline(xintercept = x_pos, linetype = "dotted",size=1) +
   facet_wrap(~variable,ncol=1,scales='free_y') +
   theme_bw() +
   theme(
       strip.text = element_text(size = 20),
       axis.title=element_text(size=20),
       axis.text.x = element_text(size = 16,angle = 45, vjust = 1, hjust=1),
       axis.text.y = element_text(size = 16),
       axis.title.x = element_text(size = 16),
       axis.title.y = element_text(size = 16),
       legend.text=element_text(size=16),
       legend.title=element_text(size=18),
   ) +
   ylab(expression(alpha))+ ylim(-0.1,0.7) + scale_fill_manual(values=clrs$color,labels=clrs$cell) + guides(fill="none")

for(i in 1:10){
    if(i == 1){
        p = p + annotate("text", x = x_pos[1]/2, y =0.7,label=unique(tmp$subline)[i])
    }else if(i==10){
        p = p + annotate("text", x = 71.5 - (71.5 - x_pos[i-1])/2, y =0.7,label=unique(tmp$subline)[i])
    }else{
        p = p + annotate("text", x = x_pos[i] - (x_pos[i] - x_pos[i-1])/2, y =0.7,label=unique(tmp$subline)[i])
    }
}

p

ggsave(paste0(path,"abcmk/Developmental/developmental_abcmk_top_500_orthologs.svg"),height=10,width=30)

df_case_alpha = merge(df_ci[,1:5],df[type=='case',c(1:2,21,20,19)],all=T)

df_case_alpha = df_case_alpha[order(line,cell)]
df_case_alpha[,6:8] = round(df_case_alpha[,6:8],3)
fwrite(df_case_alpha,paste0(path,"/abcmk/Developmental/developmental_abcmk_alpha_ci_pvalues.txt"),sep='\t')

fwrite(df[type=="case"],paste0(path,"/abcmk/Developmental/developmental_abcmk_alpha_pvalues.txt"),sep='\t')

##################
df = empirical_p(df,"ωₐ")

d = melt(df[,c(1:3,7:9,16:19)],id.vars=c("line","cell","subline","type","n","pvalue","color"))

d$cell = gsub("_","-",d$cell)
d$cell = gsub("DNearly","DN(early)",d$cell)
d$cell = gsub("DNP","DN(P)",d$cell)
d$cell = gsub("DNQ","DN(Q)",d$cell)
d$cell = gsub("DPP","DP(P)",d$cell)


d$cell = gsub("-HSC-progenitors"," (HSC-progenitors)",d$cell)
d$cell = gsub("-Myeloid-ALL"," (Myeloid-ALL)",d$cell)


d = d[order(variable,decreasing=T)]

d_p= melt(df[,c(1:3,16:21)],id.vars=c("line","cell","subline","type","n","color"))

d_p$cell = gsub("_","-",d_p$cell)
d_p$cell = gsub("DNearly","DN(early)",d_p$cell)
d_p$cell = gsub("DNP","DN(P)",d_p$cell)
d_p$cell = gsub("DNQ","DN(Q)",d_p$cell)
d_p$cell = gsub("DPP","DP(P)",d_p$cell)
d_p$cell = gsub("-HSC-progenitors"," (HSC-progenitors)",d_p$cell)
d_p$cell = gsub("-Myeloid-ALL"," (Myeloid-ALL)",d_p$cell)

d_p$label=""
d_p[type=='case'] = d_p[type=='case'] %<>% mutate(
        label = case_when(
            value > 0.05 ~ "",
            value > 0.01 ~ "*",
            value > 0.001 ~ "**",
            value > 0.0001 ~ "***",
            value > 0.00001 ~ "****",
            !is.na(value) ~ "",
            TRUE ~ NA_character_
        )
    )

tmp = d[type == "case" & variable == "ωₐ"]
tmp = split(tmp,tmp$subline)

subline_order <- c("HSC_progenitors", "Tcells", "Bcells", "Innate", "MACs","Granulocytes","DCs","Stromal","MegaKaryocytes","Erythrocytes")

tmp = tmp[subline_order]

for(i in names(tmp)){
    tmp[[i]] = tmp[[i]][order(value,decreasing=T)]
}

tmp = rbindlist(tmp)
x = tmp$cell

d$variable = factor(d$variable,levels=c("ωₐ","ωₐ_strong","ωₐ_weak"))
d$cell = factor(d$cell,levels=x)
d_p$cell = factor(d_p$cell,levels=x)
d$pvalue = d_p$value
d$label = d_p$label

d_flt = d[!grepl("all",d$cell)]
cols = c("line","cell")
d_flt = d_flt %>% group_by(across(all_of(cols))) %>% mutate("n"=n()) %>% as.data.table
d_flt = d_flt[n>3]

x_pos = c(12.5,22.5,31.5,36.5,47.5,51.5,58.5,62.5,67.5)
clrs = d_flt[type=='case',c('cell','color')] %>% unique
clrs = clrs[order(cell)]

p = ggplot(d_flt[type=="control"],aes(x=cell,y=value,fill=cell)) +
   geom_violin(width=0.8) +
   geom_errorbar(data=d_flt[type=="case"],aes(x = cell, ymin = value,ymax = value),size=1,color="black") +
   geom_text(data=d_flt[type=="case"],aes(x=cell,y=-0.025,label = label,color=line),size=10,color="black",angle = 90, vjust = 0.75, hjust=1) +
   geom_vline(xintercept = x_pos, linetype = "dotted",size=1) +
   facet_wrap(~variable,ncol=1,scales='free_y') +
   theme_bw() +
   theme(
       strip.text = element_text(size = 20),
       axis.title=element_text(size=20),
       axis.text.x = element_text(size = 16,angle = 45, vjust = 1, hjust=1),
       axis.text.y = element_text(size = 16),
       axis.title.x = element_text(size = 16),
       axis.title.y = element_text(size = 16),
       legend.text=element_text(size=16),
       legend.title=element_text(size=18),
   ) +
   ylab(expression(omega[a]))+ ylim(-0.06,0.3) + scale_fill_manual(values=clrs$color,labels=clrs$cell) + guides(fill="none")

for(i in 1:10){
    if(i == 1){
        p = p + annotate("text", x = x_pos[1]/2, y =0.3,label=unique(tmp$subline)[i])
    }else if(i==10){
        p = p + annotate("text", x = 71.5 - (71.5 - x_pos[i-1])/2, y =0.3,label=unique(tmp$subline)[i])
    }else{
        p = p + annotate("text", x = x_pos[i] - (x_pos[i] - x_pos[i-1])/2, y =0.3,label=unique(tmp$subline)[i])
    }
}

ggsave(paste0(path,"/abcmk/Developmental/developmental_abcmk_omega_top_500_orthologs.svg"),height=10,width=30)

df_case_omega = merge(df_ci[,c(1,2,6:8)],df[type=='case',c(1:2,21,20,19)],all=T)

df_case_omega = df_case_omega[order(line,cell)]
df_case_omega[,6:8] = round(df_case_omega[,6:8],3)


fwrite(df_case_omega,paste0(path,"/abcmk/Developmental/developmental_abcmk_omega_ci_pvalues.txt"),sep='\t')

fwrite(df[type=="case"],paste0(path,"/abcmk/Developmental/developmental_abcmk_omega_pvalues.txt"),sep='\t')

"""

# Adult tissues

In [ ]:
lines = readdir(labstorage * "Adult/ENS_FULL_genelists_wilcox",join=true)

mkpath(labstorage * "/abcmk/Adult")

mkpath.(labstorage .* "/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/" .* ["datasets","outputs"])

@rput lines

## Bootstrap

In [ ]:
genes = CSV.read(labstorage * "annotations/MKdata_may2023.txt",DataFrame,header=false);
orthologs = CSV.read(labstorage * "annotations/mammals_orthologs.txt",DataFrame,header=true);

rename!(orthologs,:mgPropSitesAdapt => :ids);

In [ ]:
df_prot = CSV.read(labstorage * "/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/ensembl_protcoding_genes_v109",DataFrame,header=false)
insertcols!(df_prot,2,:c=>"no")       
# DataFrame to be changed
# Change directory before to run bootstrap
cd("/home/jmurgamoreno/Immune_Adaptation_Atlas_2023/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/")

In [ ]:
for l in lines

    files = filter(x -> occursin("FULL_ENSEMBL.txt",x) ,readdir(l,join=true))

    for f ∈ files
        @show f

        f_path,f_de = splitdir(f)

        f_de = replace(f_de,"_FULL_ENSEMBL"=>"")
        f_de = split(f_path,"/")[end] * "_" * split(f_de,".")[1]

        # Extracting the top 500 DE genes by cell line.
        df = CSV.read(f,DataFrame,header=false)

        # Get orthologs only
        rename!(df,:Column1=>:ids)

        df_orthologs = innerjoin(df,orthologs,on=:ids,order=:left)

        top_de = first(df_orthologs,500)
        df_prot[in.(df_prot.Column1,[top_de.ids]),:c] .= "yes"

        CSV.write(labstorage * "/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/datasets/" * f_de * ".txt",df_prot,header=false,delim=" ")

        # Run perl bootstrap
        cmd = `bash -c "./run.sh $f_de"`
        run(cmd)
        
        # Reset case df 
        df_prot[!,:c] .= "no"

    end
end

mv(labstorage * "/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/datasets/",labstorage * "/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/Adult")
mv(labstorage * "/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/outputs/",labstorage * "/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/Adult_outputs")

## ABC-MK

In [ ]:
adap = MKtest.parameters(n=661,dac=[2,4,5,10,20,50,200,661],cutoff=[0.0,0.7]);

In [ ]:
out_adult = []
param = adap = MKtest.parameters(n=661,dac=[2,4,5,10,20,50,200,661],cutoff=[0.0,0.7]);
files = filter(x-> !isdir(x),readdir(labstorage * "/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/Adult_outputs/",join=true))
@showprogress for i in files
    abcmk = cell_analysis(adap,file=i,data_tgp=labstorage * "/annotations/MKdata_may2023.txt",rates=labstorage * "/abcmk/rates_hpc.jld2")
    @show size(abcmk[1])

    push!(out_adult,abcmk)
end


In [ ]:
# Save dict results
JLD2.jldopen(path * "/abcmk/results_immune_abc.jld2", "a+") do file
    file["Adult"] = out_adult
end

## Clean and plot

In [ ]:

df_adult = vcat(map(x->x[1],out_adult)...);
df_adult.line .= map(x-> join(split(x,"_")[1:1],"_"),df_adult.cell);
df_adult.cell = replace.(df_adult.cell,"Bcells_"=>"");
df_adult.cell = replace.(df_adult.cell,"Tcells_"=>"");
df_adult.cell = replace.(df_adult.cell,"Myeloid_"=>"");

df_adult = insertcols(df_adult,16,:n=>0);
df_adult[df_adult.type .== "case",:n] .= countlines.(files[occursin.("case",files)]);
df_adult = sort(df_adult,[:line,:cell]);

df_ci_adult = vcat(map(x-> x[2] ,out_adult[occursin.("case",files)])...);
insertcols!(df_ci_adult,1,:cell=>map(x-> splitdir(x)[end],files[occursin.("case",files)]));

insertcols!(df_ci_adult,1,:line=>map(x-> join(split(splitdir(x)[end],"_")[1:1],"_"),files[occursin.("case",files)]));

df_ci_adult = sort(df_ci_adult,[:line,:cell]);
df_ci_adult.cell = replace.(df_ci_adult.cell,"Myeloid_"=>"");
df_ci_adult.cell = replace.(df_ci_adult.cell,"Tcells_"=>"");
df_ci_adult.cell = replace.(df_ci_adult.cell,"Bcells_"=>"");
df_ci_adult.cell = replace.(df_ci_adult.cell,"_case.txt"=>"");

df_ci_adult = innerjoin(df_adult[df_adult.type .=="case",[:line,:cell,:n]],df_ci_adult,on=[:line,:cell]);

CSV.write(path * "/abcmk/Adult/adult_abcmk_inference_alpha_top_500_orthologs.txt",df_adult);
CSV.write(path * "/abcmk/Adult/adult_abcmk_ci_alpha_top_500_orthologs.txt",df_ci_adult);

@rput df_adult
@rput df_ci_adult
@rput path

In [ ]:
R"""
library(magrittr)
library(paletteer)
library(data.table)
library(dplyr)
library(tidyr)
library(ggplot2)

empirical_p <- function(df,symbol){
    x = split(df,df$cell)

    out = list()
    for(i in names(x)){
        x[[i]]$pvalue = rank(x[[i]][[symbol]] * -1,na.last = "keep")/sum(is.na (x[[i]][[symbol]]) == F)
        x[[i]]$pvalue_strong = rank(x[[i]][[paste0(symbol,"_strong")]] * -1,na.last = "keep")/sum(is.na (x[[i]][[paste0(symbol,"_strong")]]) == F)
        x[[i]]$pvalue_weak = rank(x[[i]][[paste0(symbol,"_weak")]] * -1,na.last = "keep")/sum(is.na (x[[i]][[paste0(symbol,"_weak")]]) == F)
        out[[i]] = x[[i]]
    }
    return(rbindlist(out))
}

df = as.data.table(df_adult) 
df_ci = as.data.table(df_ci_adult)
df = df[cell != "ALL_DE"]
df_ci = df_ci[cell != "ALL_DE"]
df$cell = gsub("_case.txt","",df$cell)
df$cell = gsub("_control.txt","",df$cell)
df$cell = gsub("_FULL_ENSEMBL_TOP","",df$cell)

tcells = data.table(cell=df[line=='Tcells']$cell %>%unique,color=paletteer_c("ggthemes::Blue",df[line=='Tcells']$cell %>% unique %>% length))
bcells = data.table(cell=df[line=='Bcells']$cell %>% unique,color=paletteer_c("ggthemes::Green",df[line=='Bcells']$cell %>% unique %>% length))
macs = data.table(cell=df[line=='Myeloid']$cell %>% unique,color=paletteer_c("ggthemes::Purple",df[line=='Myeloid']$cell %>% unique %>% length))
pal = rbindlist(list(tcells,bcells,macs))

df = empirical_p(df,"α")

df = merge(df,pal)

df_l = df[cell==line]
df = df[cell!=line]
df = df[cell!='T_CD4_CD8']
df = df[cell!='Pro-B']
df_ci = df_ci[cell!='T_CD4_CD8']
df_ci = df_ci[cell!='Pro-B']

d = melt(df[,c(1:6,16:17,20)],id.vars=c("line","cell","type","n","pvalue","color"))

d = d[order(variable,decreasing=T)]

d_p = melt(df[,c(1:3,16:20)],id.vars=c("line","cell","type","n","color"))

d_p$variable='pvalue'

d_p$label=""
d_p[type=='case'] = d_p[type=='case'] %<>% mutate(
        label = case_when(
            value > 0.05 ~ "",
            value > 0.01 ~ "*",
            value > 0.001 ~ "**",
            value > 0.0001 ~ "***",
            value > 0.00001 ~ "****",
            !is.na(value) ~ "",
            TRUE ~ NA_character_
        )
    )


tmp = d[type == "case" & variable == "α"]
tmp = split(tmp,tmp$line)

line_order <- c("Tcells", "Bcells","Myeloid")

tmp = tmp[line_order]

for(i in names(tmp)){
    tmp[[i]] = tmp[[i]][order(value,decreasing=T)]
}

tmp = rbindlist(tmp)
x = tmp$cell

d$variable = factor(d$variable,levels=c("α","α_strong","α_weak"))
d$cell = factor(d$cell,levels=x)
d_p$cell = factor(d_p$cell,levels=x)
d$pvalue = d_p$value
d$label = d_p$label

d_flt = d[!grepl("all",d$cell)]
cols = c("line","cell")
d_flt = d_flt %>% group_by(across(all_of(cols))) %>% mutate("n"=n()) %>% as.data.table
# d_flt = d_flt[n>3]

x_pos = c(16.5,24.5)
clrs = d_flt[type=='case',c('cell','color')] %>% unique
clrs = clrs[order(cell)]
p = ggplot(d_flt[type=="control"],aes(x=cell,y=value,fill=cell)) +
   geom_violin(width=0.8) + 
   geom_errorbar(data=d_flt[type=="case"],aes(x = cell, ymin = value,ymax = value),size=1,color="black") + 
   geom_text(data=d_flt[type=="case"],aes(x=cell,y=-0.05,label = label,color=line),size=14,color="red",angle = 90, vjust = 0.75, hjust=1) +
   geom_vline(xintercept = x_pos, linetype = "dotted",size=1) + 
   facet_wrap(~variable,ncol=1,scales='free_y') + 
   theme_bw() +
   theme(
       strip.text = element_text(size = 20),
       axis.title=element_text(size=20),
       axis.text.x = element_text(size = 16,angle = 45, vjust = 1, hjust=1),
       axis.text.y = element_text(size = 16), 
       axis.title.x = element_text(size = 16),  
       axis.title.y = element_text(size = 16),
       legend.text=element_text(size=16),
       legend.title=element_text(size=18), 
   ) +
   ylab(expression(alpha)) + scale_fill_manual(values=clrs$color,labels=clrs$cell) + guides(fill="none") + ylim(-0.2,0.65)

for(i in 1:3){
    if(i == 1){
        p = p + annotate("text", x = x_pos[1]/2, y =0.7,label=unique(tmp$line)[i])
    }else if(i==3){
        p = p + annotate("text", x = 33.5 - (33.5 - x_pos[i-1])/2, y =0.7,label=unique(tmp$line)[i])
    }else{
        p = p + annotate("text", x = x_pos[i] - (x_pos[i] - x_pos[i-1])/2, y =0.7,label=unique(tmp$line)[i])
    }
}

p

df_case_alpha = merge(df_ci[,c(1:2,4:6)],df[type=='case',c(1:2,19,17,18)],all=T)
df_case_alpha[,6:8] = round(df_case_alpha[,6:8],3)

ggsave(paste0(path,"/abcmk/Adult/adult_abcmk_top_500_orthologs.svg"),height=10,width=15)

fwrite(df_case_alpha,paste0(path,"/abcmk/Adult/adult_abcmk_alpha_ci_pvalues.txt"),sep='\t')

fwrite(df[type=="case"],paste0(path,"/abcmk/Adult/adult_abcmk_alpha_pvalues.txt"),sep='\t')

###

df = empirical_p(df,"ωₐ")

d = melt(df[,c(1:3,7:9,16:17,20)],id.vars=c("line","cell","type","n","pvalue","color"))

d = d[order(variable,decreasing=T)]

d_p= melt(df[,c(1:3,16:20)],id.vars=c("line","cell","type","n","color"))

d_p$variable='pvalue'

d_p$label=""
d_p[type=='case'] = d_p[type=='case'] %<>% mutate(
        label = case_when(
            value > 0.05 ~ "",
            value > 0.01 ~ "*",
            value > 0.001 ~ "**",
            value > 0.0001 ~ "***",
            value > 0.00001 ~ "****",
            !is.na(value) ~ "",
            TRUE ~ NA_character_
        )
    )


tmp = d[type == "case" & variable == "ωₐ"]
tmp = split(tmp,tmp$line)

line_order <- c("Tcells", "Bcells","Myeloid")

tmp = tmp[line_order]

for(i in names(tmp)){
    tmp[[i]] = tmp[[i]][order(value,decreasing=T)]
}

tmp = rbindlist(tmp)
x = tmp$cell

d$variable = factor(d$variable,levels=c("ωₐ","ωₐ_strong","ωₐ_weak"))
d$cell = factor(d$cell,levels=x)
d_p$cell = factor(d_p$cell,levels=x)
d$pvalue = d_p$value
d$label = d_p$label

d_flt = d[!grepl("all",d$cell)]
cols = c("line","cell")
d_flt = d_flt %>% group_by(across(all_of(cols))) %>% mutate("n"=n()) %>% as.data.table

x_pos = c(16.5,24.5)
clrs = d_flt[type=='case',c('cell','color')] %>% unique
clrs = clrs[order(cell)]
p = ggplot(d_flt[type=="control"],aes(x=cell,y=value,fill=cell)) +
   geom_violin(width=0.8) +
   geom_errorbar(data=d_flt[type=="case"],aes(x = cell, ymin = value,ymax = value),size=1,color="black") +
   geom_text(data=d_flt[type=="case"],aes(x=cell,y=-0.025,label = label,color=line),size=10,color="black",angle = 90, vjust = 0.75, hjust=1) +
   geom_vline(xintercept = x_pos, linetype = "dotted",size=1) +
   facet_wrap(~variable,ncol=1,scales='free_y') +
   theme_bw() +
   theme(
       strip.text = element_text(size = 20),
       axis.title=element_text(size=20),
       axis.text.x = element_text(size = 16,angle = 45, vjust = 1, hjust=1),
       axis.text.y = element_text(size = 16),
       axis.title.x = element_text(size = 16),
       axis.title.y = element_text(size = 16),
       legend.text=element_text(size=16),
       legend.title=element_text(size=18),
   ) +
   ylab(expression(omega[a]))+ ylim(-0.1,0.3) + scale_fill_manual(values=clrs$color,labels=clrs$cell) + guides(fill="none")

for(i in 1:3){
    if(i == 1){
        p = p + annotate("text", x = x_pos[1]/2, y =0.3,label=unique(tmp$line)[i])
    }else if(i==3){
        p = p + annotate("text", x = 33.5 - (33.5 - x_pos[i-1])/2, y =0.3,label=unique(tmp$line)[i])
    }else{
        p = p + annotate("text", x = x_pos[i] - (x_pos[i] - x_pos[i-1])/2, y =0.3,label=unique(tmp$line)[i])
    }
}

df_case_omega = merge(df_ci[,c(1,2,7:9)],df[type=='case',c(1:2,19,17,18)],all=T)
df_case_omega[,6:8] = round(df_case_omega[,6:8],3)

ggsave(paste0(path,"abcmk/Adult/adult_abcmk_omega_top_500_orthologs.svg"),height=10,width=15)

fwrite(df_case_omega,paste0(path,"abcmk/Adult/adult_abcmk_omega_ci_pvalues.txt"),sep='\t')

fwrite(df[type=="case"],paste0(path,"abcmk/Adult/adult_abcmk_omega_pvalues.txt"),sep='\t')

"""

# Lung

In [ ]:
lines = readdir(labstorage * "Lung/ENS_FULL_genelists_wilcox",join=true)

mkpath(labstorage * "/abcmk/Lung")

mkpath.(labstorage .* "/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/" .* ["datasets","outputs"])

@rput lines

## Bootstrap

In [ ]:
genes = CSV.read(labstorage * "annotations/MKdata_may2023.txt",DataFrame,header=false);
orthologs = CSV.read(labstorage * "annotations/mammals_orthologs.txt",DataFrame,header=true);

rename!(orthologs,:mgPropSitesAdapt => :ids);

In [ ]:
df_prot = CSV.read(labstorage * "/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/ensembl_protcoding_genes_v109",DataFrame,header=false)
insertcols!(df_prot,2,:c=>"no")       
# DataFrame to be changed
# Change directory before to run bootstrap
cd("/home/jmurgamoreno/Immune_Adaptation_Atlas_2023/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/")

In [ ]:
for l in lines

    files = filter(x -> occursin("ENSEMBL.txt",x) ,readdir(l,join=true))

    for f ∈ files
        @show f

        f_path,f_de = splitdir(f)

        f_de = replace(f_de,"_ENSEMBL"=>"")
        f_de = split(f_path,"/")[end] * "_" * split(f_de,".")[1]

        # Extracting the top 500 DE genes by cell line.
        df = CSV.read(f,DataFrame,header=false)

        # Get orthologs only
        rename!(df,:Column1=>:ids)

        df_orthologs = innerjoin(df,orthologs,on=:ids,order=:left)

        top_de = first(df_orthologs,500)
        df_prot[in.(df_prot.Column1,[top_de.ids]),:c] .= "yes"

        CSV.write(labstorage * "/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/datasets/" * f_de * ".txt",df_prot,header=false,delim=" ")
        
        alpha,sfs,divergence = MKtest.parse_sfs(adap,data=labstorage * "/annotations/MKdata_may2023.txt",gene_list=labstorage * "/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/datasets/" * f_de * ".txt")

        if (size(top_de,1) < 100)
            if(sum(sfs[1][:,2:3]) < 500)
                @warn (f,size(top_de,1))
                rm(labstorage * "/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/datasets/" * f_de * ".txt")
                continue
            end
        end

        # Run perl bootstrap
        cmd = `bash -c "./run.sh $f_de"`
        run(cmd)
        
        # Reset case df 
        df_prot[!,:c] .= "no"

    end
end

mv(labstorage * "/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/datasets/",labstorage * "/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/Lung")
mv(labstorage * "/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/outputs/",labstorage * "/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/Lung_outputs")

## ABC-MK

In [ ]:
adap = MKtest.parameters(n=661,dac=[2,4,5,10,20,50,200,661],cutoff=[0.0,0.7]);

In [ ]:
out_lung = []
param = adap = MKtest.parameters(n=661,dac=[2,4,5,10,20,50,200,661],cutoff=[0.0,0.7]);
files = filter(x-> !isdir(x),readdir(labstorage * "/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/Lung_outputs/",join=true))
@showprogress for i in files[1:4]
    abcmk = cell_analysis(adap,file=i,data_tgp=labstorage * "/annotations/MKdata_may2023.txt",rates=labstorage * "/abcmk/rates_hpc.jld2")
    @show size(abcmk[1])

    push!(out_lung,abcmk)
end


In [ ]:
# Save dict results
JLD2.jldopen(path * "results/results_immune_abc.jld2", "a+") do file
    file["Lung"] = out_lung
end

## Clean and plot

In [ ]:
df_lung = vcat(map(x->x[1],out_lung)...);
df_lung = insertcols(df_lung,16,:n=>0);
df_lung[df_lung.type .== "case",:n] .= countlines.(files[occursin.("case",files)]);
df_lung = df_lung[df_lung.cell .!= "Tcells_CD8_TRM",:];

df_lung.line .= map(x-> join(split(x,"_")[1:1],"_"),df_lung.cell);
df_lung.cell = replace.(df_lung.cell,"Bcells_"=>"");
df_lung.cell = replace.(df_lung.cell,"Tcells_"=>"");
df_lung.cell = replace.(df_lung.cell,"Myeloid_"=>"");


df_lung = sort(df_lung,[:line,:cell]);
df_lung.cell = replace.(df_lung.cell,"merged_"=>"");


df_ci_lung = vcat(map(x-> x[2] ,out_lung[occursin.("case",files)])...);

insertcols!(df_ci_lung,1,:cell=>map(x-> splitdir(x)[end],files[occursin.("case",files)]));

insertcols!(df_ci_lung,1,:line=>map(x-> join(split(splitdir(x)[end],"_")[1:1],"_"),files[occursin.("case",files)]));
df_ci_lung = df_ci_lung[df_ci_lung.cell .!= "Tcells_CD8_TRM_case.txt",:];

df_ci_lung = sort(df_ci_lung,[:line,:cell]);
df_ci_lung.cell = replace.(df_ci_lung.cell,"Myeloid_"=>"");
df_ci_lung.cell = replace.(df_ci_lung.cell,"Tcells_"=>"");
df_ci_lung.cell = replace.(df_ci_lung.cell,"Bcells_"=>"");
df_ci_lung.cell = replace.(df_ci_lung.cell,"_case.txt"=>"");
df_ci_lung.cell = replace.(df_ci_lung.cell,"merged_"=>"");


CSV.write(labstorage * "/abcmk/Lung/lung_abcmk_inference_alpha_top_500_orthologs.txt",df_lung);
CSV.write(labstorage * "/abcmk/Lung/lung_abcmk_ci_alpha_top_500_orthologs.txt",df_ci_lung);

@rput df_lung;
@rput df_ci_lung
@rput path

In [ ]:
R"""
library(magrittr)
library(paletteer)
library(data.table)
library(dplyr)
library(tidyr)
library(ggplot2)

empirical_p <- function(df,symbol){
    x = split(df,df$cell)

    out = list()
    for(i in names(x)){
        x[[i]]$pvalue = rank(x[[i]][[symbol]] * -1,na.last = "keep")/sum(is.na (x[[i]][[symbol]]) == F)
        x[[i]]$pvalue_strong = rank(x[[i]][[paste0(symbol,"_strong")]] * -1,na.last = "keep")/sum(is.na (x[[i]][[paste0(symbol,"_strong")]]) == F)
        x[[i]]$pvalue_weak = rank(x[[i]][[paste0(symbol,"_weak")]] * -1,na.last = "keep")/sum(is.na (x[[i]][[paste0(symbol,"_weak")]]) == F)
        out[[i]] = x[[i]]
    }
    return(rbindlist(out))
}
df = as.data.table(df_lung)
df_ci = as.data.table(df_ci_lung)
# df[line == 'Tcells_merged']$cell = paste0(df[line == 'Tcells_merged']$cell,"_merged")
# df[line == 'Tcells_merged']$line = "Tcells"
# df$cell = gsub("_case.txt","",df$cell)
# df$cell = gsub("_control.txt","",df$cell)
# df$cell = gsub("_FULL_ENSEMBL_TOP","",df$cell)

tcells = data.table(cell=df[line=='Tcells']$cell %>%unique,color=paletteer_c("ggthemes::Blue",df[line=='Tcells']$cell %>% unique %>% length))
bcells = data.table(cell=df[line=='Bcells']$cell %>% unique,color=paletteer_c("ggthemes::Green",df[line=='Bcells']$cell %>% unique %>% length))
macs = data.table(cell=df[line=='Myeloid']$cell %>% unique,color=paletteer_c("ggthemes::Purple",df[line=='Myeloid']$cell %>% unique %>% length))

pal = rbindlist(list(tcells,bcells,macs))


df = empirical_p(df,"α")

df = merge(df,pal)


# df_l = df[cell==line]
df = df[cell!=line]
# df = df[cell!='T_CD4_CD8']

d = melt(df[,c(1:6,16:17,20)],id.vars=c("line","cell","type","n","pvalue","color"))

d = d[order(variable,decreasing=T)]

d_p = melt(df[,c(1:3,16:20)],id.vars=c("line","cell","type","n","color"))

d_p$variable='pvalue'

d_p$label=""
d_p[type=='case'] = d_p[type=='case'] %<>% mutate(
        label = case_when(
            value > 0.05 ~ "",
            value > 0.01 ~ "*",
            value > 0.001 ~ "**",
            value > 0.0001 ~ "***",
            value > 0.00001 ~ "****",
            !is.na(value) ~ "",
            TRUE ~ NA_character_
        )
    )


tmp = d[type == "case" & variable == "α"]
tmp = split(tmp,tmp$line)

line_order <- c("Tcells", "Bcells","Myeloid")

tmp = tmp[line_order]

for(i in names(tmp)){
    tmp[[i]] = tmp[[i]][order(value,decreasing=T)]
}

tmp = rbindlist(tmp)
x = tmp$cell

d$variable = factor(d$variable,levels=c("α","α_strong","α_weak"))
d$cell = factor(d$cell,levels=x)
d_p$cell = factor(d_p$cell,levels=x)
d$pvalue = d_p$value
d$label = d_p$label

d_flt = d[!grepl("all",d$cell)]
cols = c("line","cell")
d_flt = d_flt %>% group_by(across(all_of(cols))) %>% mutate("n"=n()) %>% as.data.table
# d_flt = d_flt[n>3]

x_pos = c(8.5,13.5)
clrs = d_flt[type=='case',c('cell','color')] %>% unique
clrs = clrs[order(cell)]
p = ggplot(d_flt[type=="control"],aes(x=cell,y=value,fill=cell)) +
   geom_violin(width=0.8) +
   geom_errorbar(data=d_flt[type=="case"],aes(x = cell, ymin = value,ymax = value),size=1,color="black") +
   geom_text(data=d_flt[type=="case"],aes(x=cell,y=-0.05,label = label,color=line),size=14,color="red",angle = 90, vjust = 0.75, hjust=1) +
   geom_vline(xintercept = x_pos, linetype = "dotted",size=1) +
   facet_wrap(~variable,ncol=1,scales='free_y') +
   theme_bw() +
   theme(
       strip.text = element_text(size = 20),
       axis.title=element_text(size=20),
       axis.text.x = element_text(size = 16,angle = 45, vjust = 1, hjust=1),
       axis.text.y = element_text(size = 16),
       axis.title.x = element_text(size = 16),
       axis.title.y = element_text(size = 16),
       legend.text=element_text(size=16),
       legend.title=element_text(size=18),
   ) +
   ylab(expression(alpha))+ ylim(-0.2,0.7) + scale_fill_manual(values=clrs$color,labels=clrs$cell) + guides(fill="none")

for(i in 1:3){
    if(i == 1){
        p = p + annotate("text", x = x_pos[1]/2, y =0.7,label=unique(tmp$line)[i])
    }else if(i==3){
        p = p + annotate("text", x = 23.5 - (23.5 - x_pos[i-1])/2, y =0.7,label=unique(tmp$line)[i])
    }else{
        p = p + annotate("text", x = x_pos[i] - (x_pos[i] - x_pos[i-1])/2, y =0.7,label=unique(tmp$line)[i])
    }
}
p


ggsave(paste0(path,"/abcmk/Lung/lung_abcmk_top_500_orthologs.svg"),height=10,width=15)

df_case_alpha = merge(df_ci[,1:5],df[type=='case',c(1:2,19,17,18)],all=T)

fwrite(df_case_alpha,paste0(path,"/abcmk/Lung/lung_abcmk_alpha_ci_pvalues.txt"),sep='\t')

fwrite(df[type=="case"],paste0(path,"/abcmk/Lung/lung_abcmk_alpha_pvalues.txt"),sep='\t')


###
df = as.data.table(df_lung)
df_ci = as.data.table(df_ci_lung)


tcells = data.table(cell=df[line=='Tcells']$cell %>%unique,color=paletteer_c("ggthemes::Blue",df[line=='Tcells']$cell %>% unique %>% length))
bcells = data.table(cell=df[line=='Bcells']$cell %>% unique,color=paletteer_c("ggthemes::Green",df[line=='Bcells']$cell %>% unique %>% length))
macs = data.table(cell=df[line=='Myeloid']$cell %>% unique,color=paletteer_c("ggthemes::Purple",df[line=='Myeloid']$cell %>% unique %>% length))

pal = rbindlist(list(tcells,bcells,macs))


df = empirical_p(df,"ωₐ")

df = merge(df,pal)


# df_l = df[cell==line]
df = df[cell!=line]
# df = df[cell!='T_CD4_CD8']

d = melt(df[,c(1:3,7:9,16:17,20)],id.vars=c("line","cell","type","n","pvalue","color"))

d = d[order(variable,decreasing=T)]

d_p = melt(df[,c(1:3,16:20)],id.vars=c("line","cell","type","n","color"))

d_p$variable='pvalue'

d_p$label=""
d_p[type=='case'] = d_p[type=='case'] %<>% mutate(
        label = case_when(
            value > 0.05 ~ "",
            value > 0.01 ~ "*",
            value > 0.001 ~ "**",
            value > 0.0001 ~ "***",
            value > 0.00001 ~ "****",
            !is.na(value) ~ "",
            TRUE ~ NA_character_
        )
    )


tmp = d[type == "case" & variable == "ωₐ"]
tmp = split(tmp,tmp$line)

line_order <- c("Tcells", "Bcells","Myeloid")

tmp = tmp[line_order]

for(i in names(tmp)){
    tmp[[i]] = tmp[[i]][order(value,decreasing=T)]
}

tmp = rbindlist(tmp)
x = tmp$cell

d$variable = factor(d$variable,levels=c("ωₐ","ωₐ_strong","ωₐ_weak"))
d$cell = factor(d$cell,levels=x)
d_p$cell = factor(d_p$cell,levels=x)
d$pvalue = d_p$value
d$label = d_p$label

d_flt = d[!grepl("all",d$cell)]
cols = c("line","cell")
d_flt = d_flt %>% group_by(across(all_of(cols))) %>% mutate("n"=n()) %>% as.data.table
# d_flt = d_flt[n>3]

x_pos = c(8.5,13.5)
clrs = d_flt[type=='case',c('cell','color')] %>% unique
clrs = clrs[order(cell)]
p = ggplot(d_flt[type=="control"],aes(x=cell,y=value,fill=cell)) +
   geom_violin(width=0.8) +
   geom_errorbar(data=d_flt[type=="case"],aes(x = cell, ymin = value,ymax = value),size=1,color="black") +
   geom_text(data=d_flt[type=="case"],aes(x=cell,y=-0.05,label = label,color=line),size=14,color="red",angle = 90, vjust = 0.75, hjust=1) +
   geom_vline(xintercept = x_pos, linetype = "dotted",size=1) +
   facet_wrap(~variable,ncol=1,scales='free_y') +
   theme_bw() +
   theme(
       strip.text = element_text(size = 20),
       axis.title=element_text(size=20),
       axis.text.x = element_text(size = 16,angle = 45, vjust = 1, hjust=1),
       axis.text.y = element_text(size = 16),
       axis.title.x = element_text(size = 16),
       axis.title.y = element_text(size = 16),
       legend.text=element_text(size=16),
       legend.title=element_text(size=18),
   ) +
   ylab(expression(alpha))+ ylim(-0.2,0.7) + scale_fill_manual(values=clrs$color,labels=clrs$cell) + guides(fill="none")

for(i in 1:3){
    if(i == 1){
        p = p + annotate("text", x = x_pos[1]/2, y =0.7,label=unique(tmp$line)[i])
    }else if(i==3){
        p = p + annotate("text", x = 23.5 - (23.5 - x_pos[i-1])/2, y =0.7,label=unique(tmp$line)[i])
    }else{
        p = p + annotate("text", x = x_pos[i] - (x_pos[i] - x_pos[i-1])/2, y =0.7,label=unique(tmp$line)[i])
    }
}
p

ggsave(paste0(path,"/abcmk/Lung/lung_abcmk_omega_top_500_orthologs.svg"),height=10,width=15)

df_case_omega = merge(df_ci[,c(1:2,6:8)],df[type=='case',c(1:2,19,17,18)],all=T)

fwrite(df_case_omega,paste0(path,"/abcmk/Lung/lung_abcmk_omega_ci_pvalues.txt"),sep='\t')

fwrite(df[type=="case"],paste0(path,"/abcmk/Lung/lung_abcmk_omega_pvalues.txt"),sep='\t')

"""

# Macrophages activation

In [ ]:
activation_files = readdir(labstorage * "Macrophages/Output_lists/pval5x10-2_sorted/",join=true)

mkpath(labstorage * "/abcmk/Macrophages")

mkpath.(labstorage .* "/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/" .* ["datasets","outputs"])

@rput activation_files

## Bootstrap macrophages

In [ ]:
genes = CSV.read(labstorage * "annotations/MKdata_may2023.txt",DataFrame,header=false);
orthologs = CSV.read(labstorage * "annotations/mammals_orthologs.txt",DataFrame,header=true);

rename!(orthologs,:mgPropSitesAdapt => :ids);

In [ ]:
df_prot = CSV.read(labstorage * "/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/ensembl_protcoding_genes_v109",DataFrame,header=false)
insertcols!(df_prot,2,:c=>"no")       
# DataFrame to be changed
# Change directory before to run bootstrap
cd("/home/jmurgamoreno/Immune_Adaptation_Atlas_2023/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/")

In [ ]:
for f ∈ activation_files
    @show f

    f_path,f_de = splitdir(f)

    f_de = replace(f_de,"_FULL_ENSEMBL"=>"")
    f_de =  split(f_de,".")[1]
    
    # Extracting the top 500 DE genes by cell line.
    df = CSV.read(f,DataFrame,header=false)

    # Get orthologs only
    rename!(df,:Column1=>:ids)

    df_orthologs = innerjoin(df,orthologs,on=:ids,order=:left)

    top_de = first(df_orthologs,500)
    df_prot[in.(df_prot.Column1,[top_de.ids]),:c] .= "yes"

    CSV.write(labstorage * "/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/datasets/" * f_de * ".txt",df_prot,header=false,delim=" ")

    # Run perl bootstrap
    cmd = `bash -c "./run.sh $f_de"`
    run(cmd)
    
    # Reset case df 
    df_prot[!,:c] .= "no"

end

mv(labstorage * "/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/datasets/",labstorage * "/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/Macrophages")
mv(labstorage * "/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/outputs/",labstorage * "/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/Macrophages_outputs")

## ABC-MK

In [ ]:
out_macrophages = []
param = adap = MKtest.parameters(n=661,dac=[2,4,5,10,20,50,200,661],cutoff=[0.0,0.7]);
files = filter(x-> !isdir(x),readdir(labstorage * "/abcmk/Gene_Set_Enrichment_Pipeline/exdef_folder/Macrophages_outputs/",join=true))
@showprogress for i in files[1:4]
    abcmk = cell_analysis(adap,file=i,data_tgp=labstorage * "/annotations/MKdata_may2023.txt",rates=labstorage * "/abcmk/rates_hpc.jld2")
    @show size(abcmk[1])

    push!(out_macrophages,abcmk)
end

In [ ]:
# Save dict results
JLD2.jldopen(path * "/abmkc/results_immune_abc.jld2", "a+") do file
    file["Macrophages"] = out_macrophages
end

## Clean and plot

In [ ]:
df_macrophages    = vcat(map(x->x[1],out_macrophages)...);
df_ci_macrophages = vcat(map(x-> x[2] ,out_macrophages[occursin.("case",files)])...);
insertcols!(df_ci_macrophages,1,:cell=>map(x-> splitdir(x)[end],files[occursin.("case",files)]));

# df_macrophages = sort(df_macrophages,[:line,:cell])

df_macrophages = insertcols(df_macrophages,16,:n=>0);
df_macrophages[df_macrophages.type .== "case",:n] .= countlines.(files[occursin.("case",files)]);

CSV.write("/home/jmurgamoreno/Gene_Set_Enrichment_Pipeline/exdef_folder/immune_results/macrophages_abcmk_inference_alpha_top_500_orthologs.txt",df_macrophages);
CSV.write("/home/jmurgamoreno/Gene_Set_Enrichment_Pipeline/exdef_folder/immune_results/macrophages_abcmk_ci_alpha_top_500_orthologs.txt",df_ci_macrophages);
@rput df_macrophages;
@rput df_ci_macrophages;
@rput path;

In [ ]:

R"""

library(magrittr)
library(paletteer)
library(data.table)
library(dplyr)
library(tidyr)
library(ggplot2)

empirical_p <- function(df,symbol,column){
    x = split(df,df[[column]])

    out = list()
    for(i in names(x)){
        x[[i]]$pvalue = rank(x[[i]][[symbol]] * -1,na.last = "keep")/sum(is.na (x[[i]][[symbol]]) == F)
        x[[i]]$pvalue_strong = rank(x[[i]][[paste0(symbol,"_strong")]] * -1,na.last = "keep")/sum(is.na (x[[i]][[paste0(symbol,"_strong")]]) == F)
        x[[i]]$pvalue_weak = rank(x[[i]][[paste0(symbol,"_weak")]] * -1,na.last = "keep")/sum(is.na (x[[i]][[paste0(symbol,"_weak")]]) == F)
        out[[i]] = x[[i]]
    }
    return(rbindlist(out))
}

df = df_macrophages %>% as.data.table
df_ci = df_ci_macrophages %>% as.data.table
pal = data.table(diff=c("all","Pro/anti-inflammatory","Pro/anti-inflammatory","Pro/anti-inflammatory","Viral mimics","Viral mimics","Bacterial mimics","Bacterial mimics","Bacterial mimics","Bacterial mimics","Neurodegeneration"),cell=c("all","IFNB","IFNG","IL4","R484","PIC","sLPS","P3C","CIL","LIL10","MBP"),clrs=c("#58b9fd","#faae98","#f98d83","#e22618","#206fb5","#134474","#fdfb60","#f7de35","#f2b21b","#f88a19","#be2b70"))

df$cell = gsub(".txt","",df$cell)

df[grepl("all",df$cell)]$cell = paste0(df[grepl("all",df$cell)]$cell,"_case")

df = df %>% separate(cell, sep = "_", into = c("cell", "time")) %>% select(-c(line)) %>% as.data.table
df_ci = df_ci %>% separate(cell, sep = "_", into = c("cell", "time")) %>% as.data.table

df = merge(df,pal)
df[time=='genes']$time = df[time=='genes']$cell

df[time=='genes']$time = df[time=='genes']$cell
df$time = tolower(df$time) 

df[cell=='Early' & time=='early']$cell = 'all'
df[cell=='Inter' & time=='inter']$cell = 'all'
df[cell=='Late' & time=='late']$cell = 'all'

df$cell_time = paste0(df$cell,"_",df$time)
df = empirical_p(df,"α","cell_time")

df$cell = gsub("_case.txt","",df$cell)
df$cell = gsub("_control.txt","",df$cell)
df$cell = gsub("_FULL_ENSEMBL_TOP","",df$cell)
df = df %>% select(-c(cell_time))
df[cell=='R484']$cell = 'R848'
df_ci[cell=='R484']$cell = 'R848'

d = melt(df[,c(1:6,16:19)],id.vars=c("cell","time","type","n","pvalue","clrs","diff"))
d = d[order(variable,decreasing=T)]

d_p = melt(df[,c(1:3,16:21)],id.vars=c("cell","time","type","n","clrs","diff"))

d = d[order(variable,decreasing=T)]
d_p$variable='pvalue'

d_p$label=""
d_p[type=='case'] = d_p[type=='case'] %<>%
    mutate(
        label = case_when(
            value > 0.05 ~ "",
            value > 0.01 ~ "*",
            value > 0.001 ~ "**",
            value > 0.0001 ~ "***",
            value > 0.00001 ~ "****",
            !is.na(value) ~ "",
            TRUE ~ NA_character_
        )
    )


tmp = d[type == "case" & variable == "α" & time == 'early']
tmp = split(tmp,tmp$diff)

line_order <- c("Pro/anti-inflammatory","Viral mimics","Bacterial mimics","Neurodegeneration","all")

tmp = tmp[line_order]

for(i in names(tmp)){
    tmp[[i]] = tmp[[i]][order(value,decreasing=T)]
}

tmp = rbindlist(tmp)
x = tmp$cell


d$variable = factor(d$variable,levels=c("α","α_strong","α_weak"))
d$cell = factor(d$cell,levels=unique(x))
d_p$cell = factor(d_p$cell,levels=unique(x))
d$pvalue = d_p$value
d$label = d_p$label

x_pos = c(3.5,5.5,9.5,10.5)
clrs = d[type=='case',c('cell','clrs')] %>% unique
clrs = clrs[match(x,cell)]
clrs = rbind(clrs,rbind(clrs,clrs))

d[cell == 'all']$cell = stringr::str_to_title(d[cell == 'all']$cell)
tmp[cell == 'all']$diff = stringr::str_to_title(tmp[cell == 'all']$diff)

clrs[cell == 'all']$cell = stringr::str_to_title(clrs[cell == 'all']$cell)

p1 = ggplot(d[type=="control"],aes(x=cell,y=value,fill=cell)) +
   geom_violin() +  
   geom_errorbar(data=d[type=="case"],aes(x = cell, ymin = value,ymax = value),size=1,color="black") + 
  geom_vline(xintercept = x_pos, linetype = "dotted",size=1) + 
   geom_text(data=d[type=="case" & cell != 'all'],aes(x=cell,y=-0.05,label = label,color=line),size=8,color="black",angle = 90, vjust = 0.75, hjust=1) +
   facet_wrap(~variable+time,ncol=3) + 
   theme_bw() +
   theme(
       strip.text = element_text(size = 20),
       axis.title=element_text(size=20),
       axis.text.x = element_text(size = 16,angle = 90, vjust = 0.5, hjust=1),
       axis.text.y = element_text(size = 16), 
       axis.title.x = element_text(size = 16),  
       axis.title.y = element_text(size = 16),
       legend.text=element_text(size=16),
       legend.title=element_text(size=18) 
   ) +
   ylab(expression(alpha))  + ylim(-0.15,1) +   scale_fill_manual(values=clrs$clrs,labels=clrs$cell) + guides(fill="none")


for(i in 1:5){
    if(i == 1){
        p1 = p1 + annotate("text", x = x_pos[1]/2, y =0.7,label=unique(tmp$diff)[i])
    }else if(i==4){
              p1 = p1 + annotate("text", x = x_pos[i] - (x_pos[i] - x_pos[i-1])/2, y =0.8,label="Cell damage",angle = 90)
    }
    else if(i==5){
        p1 = p1 + annotate("text", x = 11.5 - (11.5 - x_pos[i-1])/2, y =0.7,label=unique(tmp$diff)[i])
    }else{
        p1 = p1 + annotate("text", x = x_pos[i] - (x_pos[i] - x_pos[i-1])/2, y =0.7,label=unique(tmp$diff)[i])
    }
}


names(d)[2] = "Time"
d$Time = stringr::str_to_title(d$Time)
clrs_stml= c("#ff9494","#a973c9","#a9a9ff")
p2 = ggplot(d[type=="control"],aes(x=Time,y=value,fill=Time)) +
   geom_violin() + 
   geom_errorbar(data=d[type=="case"],aes(x = Time, ymin = value,ymax = value),size=1,color="black") + 
   geom_text(data=d[type=="case"],aes(x=Time,y=-0.05,label = label,color=line),size=14,color="red",angle = 90, vjust = 0.75, hjust=1) +
   facet_wrap(~variable+cell,nrow=3) + 
   theme_bw() +
   theme(
       strip.text = element_text(size = 20),
       axis.title=element_text(size=20),
       axis.text.x = element_text(size = 16,angle = 90, vjust = 1, hjust=1),
       axis.text.y = element_text(size = 16), 
       axis.title.x = element_text(size = 16),  
       axis.title.y = element_text(size = 16),
       legend.text=element_text(size=16),
       legend.title=element_text(size=18) 
   ) +
   ylab(expression(alpha)) +  scale_fill_manual(values=clrs_stml) + ylim(-.25,0.6)

ggsave(paste0(path,"/abcmk/Macrophages/macrophages_abcmk_top_500_orthologs.svg"),p2,height=10,width=14)

df_case_alpha = merge(df_ci[,1:5],df[type=='case',c(1:2,21,20,19)],all=T)
df_case_alpha[,6:8] = round(df_case_alpha[,6:8],3)
fwrite(df_case_alpha,paste0(path,"/abcmk/Macrophages/macrophages_abcmk_alpha_ci_pvalues.txt"),sep='\t')

fwrite(df[type=="case"],paste0(path,"/abcmk/Macrophages/macrophages_abcmk_alpha_pvalues.txt"),sep='\t')

##########

df$cell_time = paste0(df$cell,"_",df$time)
df = empirical_p(df,"ωₐ","cell_time")

df$cell = gsub("_case.txt","",df$cell)
df$cell = gsub("_control.txt","",df$cell)
df$cell = gsub("_FULL_ENSEMBL_TOP","",df$cell)
df = df %>% select(-c(cell_time))

d = melt(df[,c(1:3,7:9,16:19)],id.vars=c("cell","time","type","n","pvalue","clrs","diff"))
d = d[order(variable,decreasing=T)]

d_p = melt(df[,c(1:3,16:21)],id.vars=c("cell","time","type","n","clrs","diff"))

d = d[order(variable,decreasing=T)]
d_p$variable='pvalue'

d_p$label=""
d_p[type=='case'] = d_p[type=='case'] %<>%
    mutate(
        label = case_when(
            value > 0.05 ~ "",
            value > 0.01 ~ "*",
            value > 0.001 ~ "**",
            value > 0.0001 ~ "***",
            value > 0.00001 ~ "****",
            !is.na(value) ~ "",
            TRUE ~ NA_character_
        )
    )


tmp = d[type == "case" & variable == "ωₐ" & time == 'early']
tmp = split(tmp,tmp$diff)

line_order <- c("Pro/anti-inflammatory","Viral mimics","Bacterial mimics","Neurodegeneration","all")

tmp = tmp[line_order]

for(i in names(tmp)){
    tmp[[i]] = tmp[[i]][order(value,decreasing=T)]
}

tmp = rbindlist(tmp)
x = tmp$cell


d$variable = factor(d$variable,levels=c("ωₐ","ωₐ_strong","ωₐ_weak"))
d$cell = factor(d$cell,levels=unique(x))
d_p$cell = factor(d_p$cell,levels=unique(x))
d$pvalue = d_p$value
d$label = d_p$label

x_pos = c(3.5,5.5,9.5,10.5)
clrs = d[type=='case',c('cell','clrs')] %>% unique
clrs = clrs[match(x,cell)]
clrs = rbind(clrs,rbind(clrs,clrs))

d[cell == 'all']$cell = stringr::str_to_title(d[cell == 'all']$cell)
tmp[cell == 'all']$diff = stringr::str_to_title(tmp[cell == 'all']$diff)

clrs[cell == 'all']$cell = stringr::str_to_title(clrs[cell == 'all']$cell)

names(d)[2] = "Time"
d$Time = stringr::str_to_title(d$Time)
clrs_stml= c("#ff9494","#a973c9","#a9a9ff")
p2 = ggplot(d[type=="control"],aes(x=Time,y=value,fill=Time)) +
   geom_violin() +
   geom_errorbar(data=d[type=="case"],aes(x = Time, ymin = value,ymax = value),size=1,color="black") +
   geom_text(data=d[type=="case"],aes(x=Time,y=-0.05,label = label,color=line),size=8,color="black",angle = 90, vjust = 0.75, hjust=1) +
   facet_wrap(~variable+cell,nrow=3) +
   theme_bw() +
   theme(
       strip.text = element_text(size = 20),
       axis.title=element_text(size=20),
       axis.text.x = element_text(size = 16,angle = 90, vjust = 1, hjust=1),
       axis.text.y = element_text(size = 16),
       axis.title.x = element_text(size = 16),
       axis.title.y = element_text(size = 16),
       legend.text=element_text(size=16),
       legend.title=element_text(size=18)
   ) +
   ylab(expression(alpha)) + ylim(-0.15,0.3) + scale_fill_manual(values=clrs_stml)

ggsave(paste0(path,"/abcmk/Macrophages/macrophages_abcmk_omega_top_500_orthologs.svg",p2,height=10,width=14)

df_case_omega = merge(df_ci[,c(1:2,6:8)],df[type=='case',c(1:2,21,20,19)],all=T)
df_case_omega[,6:8] = round(df_case_omega[,6:8],3)

fwrite(df_case_omega,paste0(path,"/abcmk/Macrophages/macrophages_abcmk_omega_ci_pvalues.txt",sep='\t')

fwrite(df[type=="case"],paste0(path,"/abcmk/Macrophages/macrophages_abcmk_omega_pvalues.txt",sep='\t')

"""
